The following code is for HDGL which uses HD-Computing Operations. Clearly, Pytorch doesn't support operations such as Bit-wise Majority (For Bundling). This is why we translate 0/1 vectors to 1/-1 vectors as the Bundle Operator becomes signed addition in $\{1,-1\}^{\beta}$ space. Similarly, Binding operator which is XOR operation in $\{0,1\}^{\beta}$ space is the multiplication operation in the $\{1,-1\}^{\beta}$ space.



To summarize, below are the details for the bipolar counterparts:-


*    Space: $\{0,1\}^{\beta} \longleftrightarrow \{1,-1\}^{\beta}$
*    Bit: $0 \longleftrightarrow 1$

*    Bit: $1 \longleftrightarrow -1$
*    Bundle: Bitwise Majority $\longleftrightarrow$ Signed Addition

*    Binding: XOR $\longleftrightarrow$ Multiplication

In [1]:
%time
!nvcc --version

CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.81 µs
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
!pip uninstall torch -y
!pip uninstall torchvision -y
!pip uninstall torchaudio -y
!pip uninstall torchtext -y

Found existing installation: torch 2.5.0+cu121
Uninstalling torch-2.5.0+cu121:
  Successfully uninstalled torch-2.5.0+cu121
Found existing installation: torchvision 0.20.0+cu121
Uninstalling torchvision-0.20.0+cu121:
  Successfully uninstalled torchvision-0.20.0+cu121
Found existing installation: torchaudio 2.5.0+cu121
Uninstalling torchaudio-2.5.0+cu121:
  Successfully uninstalled torchaudio-2.5.0+cu121


In [2]:
!pip install torch==2.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch
print(torch.__version__)

2.4.0+cu121


In [4]:
!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html
import os
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import time
import numpy as np

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 MB 2.9 MB/s eta 0:00:00


In [71]:
import sys
import torch
import scipy
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix

In [72]:
class HDGL_utils_functions():

  def __init__(self, features_dimension, hash_length):
    self.random_A = torch.randn(features_dimension, hash_length)
    low = -2
    high = 2
    self.lmbda = (high - low) * torch.rand(hash_length) + low

    print("Here")

  def get_ids_labels(self, train_nodes_mask, val_nodes_mask, test_nodes_mask, labels_for_nodes):

    train_node_ids = torch.nonzero(train_nodes_mask).flatten()
    val_node_ids = torch.nonzero(val_nodes_mask).flatten()
    test_node_ids = torch.nonzero(test_nodes_mask).flatten()

    train_node_labels = labels_for_nodes[train_node_ids]
    val_node_labels = labels_for_nodes[val_node_ids]
    test_node_labels= labels_for_nodes[test_node_ids]

    return train_node_ids, train_node_labels, val_node_ids, val_node_labels, test_node_ids, test_node_labels

  def create_hash(self, features):
    r = torch.sparse.mm(features, self.random_A)
    r = r + self.lmbda
    r = (r > 0).float()
    r = self.convert_binary_to_bipolar(r)
    return r

  def convert_binary_to_bipolar(self, HD_vecs):
    return (2 * HD_vecs) -1

In [73]:
import scipy.sparse as sp

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

def mask_test_edges(adj): # From Thomas KipF GAE implementation https://github.com/tkipf/gae/blob/master/gae/preprocessing.py
    # Function to build test set with 10% positive links
    # NOTE: Splits are randomized and results might slightly deviate from reported numbers in the paper.
    # TODO: Clean up.

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()
    # Check that diag is zero:
    assert np.diag(adj.todense()).sum() == 0

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 10.))
    num_val = int(np.floor(edges.shape[0] / 20.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b, tol=5):
        rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], edges_all):
            continue
        if test_edges_false:
            if ismember([idx_j, idx_i], np.array(test_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(test_edges_false)):
                continue
        test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        idx_i = np.random.randint(0, adj.shape[0])
        idx_j = np.random.randint(0, adj.shape[0])
        if idx_i == idx_j:
            continue
        if ismember([idx_i, idx_j], train_edges):
            continue
        if ismember([idx_j, idx_i], train_edges):
            continue
        if ismember([idx_i, idx_j], val_edges):
            continue
        if ismember([idx_j, idx_i], val_edges):
            continue
        if val_edges_false:
            if ismember([idx_j, idx_i], np.array(val_edges_false)):
                continue
            if ismember([idx_i, idx_j], np.array(val_edges_false)):
                continue
        val_edges_false.append([idx_i, idx_j])

    assert ~ismember(test_edges_false, edges_all)
    assert ~ismember(val_edges_false, edges_all)
    assert ~ismember(val_edges, train_edges)
    assert ~ismember(test_edges, train_edges)
    assert ~ismember(val_edges, test_edges)

    data = np.ones(train_edges.shape[0])

    # Re-build adj matrix
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    # NOTE: these edge lists only contain single direction of edge!
    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

In [74]:
import os
import sys
import pickle as pkl
import numpy as np
import networkx as nx

def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def load_data(dataset):
    # load the data: x, tx, allx, graph
    names = ['x', 'tx', 'allx', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))
    x, tx, allx, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    return adj, features

To run experiments on different dataset, change below

In [75]:
adj, feat = load_data("cora")

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)

feat = torch.tensor(feat.toarray()).float()

src, dst = np.nonzero(adj_orig.toarray())

g = dgl.graph((src, dst))

#--------Remove Test Edges from DGL Graph object
for test_edge in test_edges:
  g.remove_edges([test_edge[0], test_edge[1]])
  g.remove_edges([test_edge[1], test_edge[0]])




#---------------row normalzie
row_sum = torch.sum(feat, dim=1, keepdim=True)
# Avoid division by zero by adding a small epsilon
epsilon = 1e-8
row_sum = torch.where(row_sum == 0, torch.tensor(epsilon, dtype=row_sum.dtype, device=row_sum.device), row_sum)

# Normalize each row by dividing by its sum
normalized_features = feat / row_sum
feat = normalized_features

#---------------row normalzie end

print("Features dimension:-", feat.size()[1])

HD_VEC_SIZE = 50000

HDC_helper = HDGL_utils_functions(features_dimension =  feat.size()[1], hash_length=HD_VEC_SIZE) # Change 20k,50k here

edge_hyper_vec =  torch.randint(0, 2, size=(1,HD_VEC_SIZE))[0]
edge_hyper_vec = HDC_helper.convert_binary_to_bipolar(edge_hyper_vec)

no_edge_hyper_vec =  torch.randint(0, 2, size=(1,HD_VEC_SIZE))[0]
no_edge_hyper_vec = HDC_helper.convert_binary_to_bipolar(edge_hyper_vec)

hd_embd_nodes = torch.zeros(g.number_of_nodes(), HD_VEC_SIZE)

<ipython-input-74-9c5632e39c61>:20: DeprecationWarning: Please import `csr_matrix` from the `scipy.sparse` namespace; the `scipy.sparse.csr` namespace is deprecated and will be removed in SciPy 2.0.0.
  objects.append(pkl.load(f, encoding='latin1'))


Features dimension:- 1433
Here




---


Learning begins here


---



In [76]:

feat = HDC_helper.create_hash(feat.to_sparse()) # Mapping features to HD-space using RHPT

sampled_neighbors = {}
g_2hop = dgl.transforms.khop_graph(g, 2)


for node_u, node_v in np.vstack((train_edges,val_edges)):
    # Get 1-hop neighbors
    one_hop_neighbors = g.successors(node_u).numpy()

    if len(one_hop_neighbors) == 0:
      continue

    # Sample 11 1-hop neighbors
    sampled_one_hop = np.random.choice(one_hop_neighbors, size=11, replace=True)

    # Get 2-hop neighbors
    two_hop_neighbors = g_2hop.successors(node_u).numpy()

    if len(two_hop_neighbors) == 0:
      continue

    # Sample 21 2-hop neighbors
    sampled_two_hop = np.random.choice(two_hop_neighbors, size=21, replace=True)

    N_1hop = sampled_one_hop.tolist()

    N_2hop = sampled_two_hop.tolist()

    r_i = torch.sum((torch.unsqueeze(feat[node_u],0)), axis=0)

    R_1hop = torch.sum((feat[N_1hop]),axis=0)

    R_2hop = torch.sum((feat[N_2hop]),axis=0)

    R_1hop = torch.sign(R_1hop)
    R_2hop = torch.sign(R_2hop)

    R_1hop = torch.roll(R_1hop,-1) #rotate once
    R_2hop = torch.roll(R_2hop,-2) #rotate twice

    z_u = r_i * R_1hop * R_2hop


    one_hop_neighbors = g.successors(node_v).numpy()

    if len(one_hop_neighbors) == 0:
      continue

    # Sample 11 1-hop neighbors
    sampled_one_hop = np.random.choice(one_hop_neighbors, size=11, replace=True)

    # Get 2-hop neighbors
    two_hop_neighbors = g_2hop.successors(node_v).numpy()

    if len(two_hop_neighbors) == 0:
      continue

    # Sample 21 2-hop neighbors
    sampled_two_hop = np.random.choice(two_hop_neighbors, size=21, replace=True)

    N_1hop = sampled_one_hop.tolist()

    N_2hop = sampled_two_hop.tolist()

    r_i = torch.sum((torch.unsqueeze(feat[node_v],0)), axis=0)

    R_1hop = torch.sum((feat[N_1hop]),axis=0)

    R_2hop = torch.sum((feat[N_2hop]),axis=0)

    R_1hop = torch.sign(R_1hop)
    R_2hop = torch.sign(R_2hop)

    R_1hop = torch.roll(R_1hop,-1) #rotate once
    R_2hop = torch.roll(R_2hop,-2) #rotate twice



    z_v = r_i * R_1hop * R_2hop

    hd_embd_nodes[node_u] = z_u
    hd_embd_nodes[node_v] = z_v


    edge_hyper_vec = edge_hyper_vec + torch.sign(z_u * z_v)




edge_hyper_vec = torch.sign(edge_hyper_vec)


In [77]:
# Find indices of negative edges (0s)
# negative_edges_indices = np.transpose(np.where(g.adjacency_matrix().to_dense() == 0))

negative_edges_indices = np.transpose(np.where(adj_orig.toarray() == 0))

# # Randomly select 100 negative edges
selected_negative_edges_indices = negative_edges_indices[np.random.choice(len(negative_edges_indices), 100, replace=False)]
val_edges_false.extend(selected_negative_edges_indices)

In [78]:

sampled_neighbors = {}
g_2hop = dgl.transforms.khop_graph(g, 2)


for node_u, node_v in val_edges_false:
    # Get 1-hop neighbors
    one_hop_neighbors = g.successors(node_u).numpy()

    if len(one_hop_neighbors) == 0:
      continue

    # Sample 11 1-hop neighbors (or all if there are fewer than 9)
    sampled_one_hop = np.random.choice(one_hop_neighbors, size=11, replace=True)

    # Get 2-hop neighbors
    two_hop_neighbors = g_2hop.successors(node_u).numpy()

    if len(two_hop_neighbors) == 0:
      continue

    # Sample 21 2-hop neighbors (or all if there are fewer than 9)
    sampled_two_hop = np.random.choice(two_hop_neighbors, size=21, replace=True)

    N_1hop = sampled_one_hop.tolist()

    N_2hop = sampled_two_hop.tolist()

    r_i = torch.sum((torch.unsqueeze(feat[node_u],0)), axis=0)

    R_1hop = torch.sum((feat[N_1hop]),axis=0)

    R_2hop = torch.sum((feat[N_2hop]),axis=0)

    R_1hop = torch.sign(R_1hop)
    R_2hop = torch.sign(R_2hop)

    R_1hop = torch.roll(R_1hop,-1) #rotate once
    R_2hop = torch.roll(R_2hop,-2) #rotate twice

    z_u = r_i * R_1hop * R_2hop


    one_hop_neighbors = g.successors(node_v).numpy()

    if len(one_hop_neighbors) == 0:
      continue

    # Sample 11 1-hop neighbors (or all if there are fewer than 9)
    sampled_one_hop = np.random.choice(one_hop_neighbors, size=11, replace=True)

    # Get 2-hop neighbors
    two_hop_neighbors = g_2hop.successors(node_v).numpy()

    if len(two_hop_neighbors) == 0:
      continue

    # Sample 21 2-hop neighbors (or all if there are fewer than 9)
    sampled_two_hop = np.random.choice(two_hop_neighbors, size=21, replace=True)

    N_1hop = sampled_one_hop.tolist()

    N_2hop = sampled_two_hop.tolist()

    r_i = torch.sum((torch.unsqueeze(feat[node_v],0)), axis=0)

    R_1hop = torch.sum((feat[N_1hop]),axis=0)

    R_2hop = torch.sum((feat[N_2hop]),axis=0)

    R_1hop = torch.sign(R_1hop)
    R_2hop = torch.sign(R_2hop)

    R_1hop = torch.roll(R_1hop,-1) #rotate once
    R_2hop = torch.roll(R_2hop,-2) #rotate twice



    z_v = r_i * R_1hop * R_2hop

    no_edge_hyper_vec = no_edge_hyper_vec + torch.sign(z_u * z_v)



no_edge_hyper_vec = torch.sign(no_edge_hyper_vec)

In [79]:
pos_edge_v = hd_embd_nodes * edge_hyper_vec
neg_edge_v = hd_embd_nodes * no_edge_hyper_vec


adj_distances_pos = torch.cdist(pos_edge_v, hd_embd_nodes)/HD_VEC_SIZE
adj_distances_neg = torch.cdist(neg_edge_v, hd_embd_nodes)/HD_VEC_SIZE


adj_probs = torch.where(adj_distances_pos < adj_distances_neg, (1-adj_distances_pos) + adj_distances_neg, adj_distances_pos - (1 -adj_distances_neg))

adj_prediction = torch.sigmoid(adj_probs)


In [80]:
from sklearn.metrics import roc_auc_score, average_precision_score

def get_roc_score(edges_pos, edges_neg, adj_pred, adj_orig):
    '''''
    adj_pred is the reconstructed adjacency matrix
    adj_orig is the original adjacency matrix
    Get AUCROC score and AP score
    This Function is based of Thomas Kipf VGAE implementation
    (https://github.com/tkipf/gae/tree/master)

    '''

    # Predict on test set of edges
    preds = []
    pos = []
    for e in edges_pos:
        preds.append((adj_pred[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])


    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append((adj_pred[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])


    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)


    return roc_score, ap_score

In [ ]:
get_roc_score(test_edges, test_edges_false, adj_prediction.numpy(), adj_orig.toarray())

(0.8645226101703459, 0.8995520738047991)

In [81]:
get_roc_score(test_edges, test_edges_false, adj_prediction.numpy(), adj_orig.toarray())

(0.8595969452235812, 0.8918983118997427)